In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt
import napari

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi, regionprops
from image_analysis.preprocess import mask

In [3]:
pixel_size = 2

In [4]:
folder_pretreated = r'/home/natalia/Documents/github_image_analysis/experimental_data/treatment_all/'
df = pd.read_excel("../images.xlsx")

In [5]:
df.head()

,Sample name,HCR,"Animal size, cm","Regeneration time, dpa","Exact animal size (snout to tail), cm","Average animal size inside each group, cm",Unnamed: 6,Unnamed: 7
0,hcr11_lb27_fl,11,development,development,NaN,NaN,NaN,NaN
1,hcr11_lb28_fl,11,development,development,NaN,NaN,NaN,NaN
2,hcr15_lb41_fl,15,development,development,NaN,NaN,NaN,NaN
3,hcr16_lb44_fl,16,development,development,NaN,NaN,NaN,NaN
4,hcr16_lb45_fl,16,development,development,NaN,NaN,NaN,NaN


# Loop trough all samples

In [6]:
for pc in [50]:

    df[f'system_size_L_along_SF_from_proj_pc{pc}_um']  = pd.NA
    df[f'length_L_SF_pc{pc}_um']  = pd.NA
    
    df[f'x0_pc{pc}_um_shh']  = pd.NA
    df[f'x0_pc{pc}_um_fgf'] = pd.NA
    
    df[f'PD_length_{pc}_um'] = pd.NA

In [7]:
ex_2d = np.array([0,1])

In [8]:
for i, file_name in enumerate( df['Sample name'].values):

    print(i, file_name)
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    for pc in [50]:

        if (f'shh{pc}_clip' in keys) and (f'fgf{pc}_clip' in keys):
            
            image_to_rotate = dict(zip(
                ['shh_mask', 'fgf_mask', 'tissue_mask_with_epi'],
                [ (image[f'shh{pc}_clip'] > 0).astype(int), (image[f'fgf{pc}_clip']>0).astype(int), image['tissue_mask_with_epi'] ]
            ))
            
            image_to_rotate['shh_mask'] = mask.largest_component( image_to_rotate['shh_mask'] )
            image_to_rotate['fgf_mask'] = mask.few_largest_components( image_to_rotate['fgf_mask'], 2 )
             
            regionprops_3d = regionprops.regionprops_3d(image_to_rotate, 1, 1, 1, pixel_size)
            regionprops_3d.center_mass(['shh_mask', 'fgf_mask'])
            regionprops_3d.tensor_inertia('tissue_mask_with_epi')
            
            if file_name not in ['hcr21_lb68_fr']:
                image_to_rotate = regionprops_3d.rotate_N_desired_to_ez()
            else:
                image_to_rotate = regionprops_3d.rotate_N_desired_to_ez_special()
            
            # projection
            for key in image_to_rotate.keys():
                image_to_rotate[key] = roi.mean_nonzero( image_to_rotate[key], 0)
                if 'mask' in key:
                    image_to_rotate[key] = (image_to_rotate[key] > 0) * 1

            # SF to be horisontal
            S = regionprops.center_mass(image_to_rotate['shh_mask'])
            F = regionprops.center_mass(image_to_rotate['fgf_mask'])
            
            SF   = F - S
            
            rot_x = 0
            rot_y = 0
            rot_z = roi.angle_between_2d(
                    SF, 
                    ex_2d
                )      
            
            image_to_rotate = roi.rotate(image_to_rotate, rot_x, rot_y, rot_z)

            S = regionprops.center_mass(image_to_rotate['shh_mask'])

            L = np.sum( 
                mask.largest_component( image_to_rotate['tissue_mask_with_epi'][S[0], :]  )
            ) * pixel_size
            L_SF  = np.abs( S[1] - F[1] ) * pixel_size
            
            ymin = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :]) )[0][0]
            ymax = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :]) )[0][-1]            
            
            x0_S = min( (S[1] - ymin) * pixel_size, (ymax - S[1]) * pixel_size )
            x0_F = min( (F[1] - ymin) * pixel_size, (ymax - F[1]) * pixel_size )

            df.loc[df['Sample name'] == file_name, f'system_size_L_along_SF_from_proj_pc{pc}_um']  = L
            df.loc[df['Sample name'] == file_name, f'length_L_SF_pc{pc}_um']             = L_SF
            df.loc[df['Sample name'] == file_name, f'x0_pc{pc}_um_shh']                  = x0_S
            df.loc[df['Sample name'] == file_name, f'x0_pc{pc}_um_fgf']                  = x0_F
            
            df.loc[df['Sample name'] == file_name, f'PD_length_{pc}_um']                 = ( image_to_rotate['tissue_mask_with_epi'].shape[0] - S[0]) * pixel_size

            
df.to_excel("results/system_size_L_and_gaps_x0_from_proj.xlsx", index=False)

0 hcr11_lb27_fl


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:30: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/D

Angle between ez and N_desired: 10
1 hcr11_lb28_fl


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:67: RuntimeWarning: Mean of empty slice
  arr = np.nanmean(arr, axis=axis_n)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:69: RuntimeWarning: invalid value encountered in cast
  return arr.astype( np.uint16 )
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:37: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(img_arr, axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:40: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(img_arr, axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarni

Angle between ez and N_desired: -5
2 hcr15_lb41_fl


...

/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:67: RuntimeWarning: Mean of empty slice
  arr = np.nanmean(arr, axis=axis_n)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:69: RuntimeWarning: invalid value encountered in cast
  return arr.astype( np.uint16 )
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:37: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(img_arr, axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:40: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(img_arr, axis=1)


48 hcr21_7cm_12dpa_noif


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:30: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


Angle between ez and N_desired: 0


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:67: RuntimeWarning: Mean of empty slice
  arr = np.nanmean(arr, axis=axis_n)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/roi.py:69: RuntimeWarning: invalid value encountered in cast
  return arr.astype( np.uint16 )
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:37: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(img_arr, axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:40: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(img_arr, axis=1)
